# Regression and forecasting with aeon

This notebook describes a built in way of forecasting using regression that
encapsulates the whole process into a forecaster.

From an interface perspective, forecasting through regression can be formulated as
"reduction". There are dangers in a simplistic approach to forecasting via regression.

In [ ]:
# general imports
import numpy as np
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split

from aeon.datasets import load_airline
from aeon.forecasting.model_selection import temporal_train_test_split
from aeon.utils.plotting import plot_series

y = load_airline()
y_train, y_test = train_test_split(y)
plot_series(y_train.sort_index(), y_test.sort_index(), labels=["y_train", "y_test"]);

ModuleNotFoundError: No module named 'matplotlib'. 'matplotlib' is a soft dependency and not included in the base aeon installation. Please run: `pip install matplotlib` to install the matplotlib package. To install all soft dependencies, run: `pip install aeon[all_extras]`

This leads to leakage: The data you are using to train a machine learning algorithm happens to have the information you are trying to predict.

But `train_test_split(y, shuffle=False)` works, which is what `temporal_train_test_split(y)` does in `aeon`:

In [ ]:
y_train, y_test = temporal_train_test_split(y)
plot_series(y_train, y_test, labels=["y_train", "y_test"]);

### Pitfall 2: obscure data manipulations, brittle boilerplate code to apply regressors

It is common practice to apply supervised regressors after transforming the data for forecasting, through lagging - for example, in auto-regressive reduction strategies.

Two important pitfalls appear right at the start:

* a lot of boilerplate code has to be written to transform the data to make it ready for fitting - this is highly error prone
* there are a number of implicit hyper-parameters here, such as window and lag size. If done without caution, these are not explicit or tracked in the experiment, which can lead to "p-value hacking".

Below is an example of such boilerplate code to demonstrate this. The code is closely modelled on the R code used in the [M4 competition](https://github.com/Mcompetitions/M4-methods):

In [ ]:
# suppose we want to predict 3 years ahead
fh = np.arange(1, 37)

In [ ]:
# slightly modified code from the M4 competition
def split_into_train_test(data, in_num, fh):
    """
    Splits the series into train and test sets.

    Each step takes multiple points as inputs
    :param data: an individual TS
    :param fh: number of out of sample points
    :param in_num: number of input points for the forecast
    :return:
    """
    train, test = data[:-fh], data[-(fh + in_num) :]
    x_train, y_train = train[:-1], np.roll(train, -in_num)[:-in_num]
    x_test, y_test = test[:-1], np.roll(test, -in_num)[:-in_num]
    #     x_test, y_test = train[-in_num:], np.roll(test, -in_num)[:-in_num]

    # reshape input to be [samples, time steps, features]
    # (N-NF samples, 1 time step, 1 feature)
    x_train = np.reshape(x_train, (-1, 1))
    x_test = np.reshape(x_test, (-1, 1))
    temp_test = np.roll(x_test, -1)
    temp_train = np.roll(x_train, -1)
    for _ in range(1, in_num):
        x_train = np.concatenate((x_train[:-1], temp_train[:-1]), 1)
        x_test = np.concatenate((x_test[:-1], temp_test[:-1]), 1)
        temp_test = np.roll(temp_test, -1)[:-1]
        temp_train = np.roll(temp_train, -1)[:-1]

    return x_train, y_train, x_test, y_test

In [ ]:
# here we split the time index, rather than the actual values,
# to show how we split the windows
feature_window, target_window, _, _ = split_into_train_test(
    np.arange(len(y)), 10, len(fh)
)

To better understand the prior data transformation, we can look at how we can split the training series into windows. Here we show the generated windows expressed as integer indices:

In [ ]:
feature_window[:5, :]

In [ ]:
target_window[:5]

In [ ]:
# now we can split the actual values of the time series
x_train, y_train, x_test, y_test = split_into_train_test(y.values, 10, len(fh))
print(x_train.shape, y_train.shape)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(x_train, y_train)

To reiterate the potential pitfalls here:

> The manual requires a lot of hand-written code which is often error-prone, not modular and not tuneable.

These steps involve a number of implicit hyper-parameters:

> * The way you slice the time series into windows (e.g. the window length);
> * The way you generate forecasts (recursive strategy, direct strategy, other hybrid strategies).

### Pitfall 3: Given a fitted regression algorithm, how can we generate forecasts?

The next important pitfall comes at the end:

If making predictions along the "manual route" for supervised regressors, the supervised regressor's outputs have to be transformed back into forecasts. This is easily forgotten, and invites errors in forecasts and evaluation (see pitfall no.1) - especially, if one does not cleanly keep track of which data is known at what time, or how to invert the transformation made in fitting.

A naive user might now proceed like this:

In [ ]:
print(x_test.shape, y_test.shape)

# add back time index to y_test
y_test = pd.Series(y_test, index=y.index[-len(fh) :])

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
from aeon.performance_metrics.forecasting import mean_absolute_percentage_error

mean_absolute_percentage_error(
    y_test, pd.Series(y_pred, index=y_test.index), symmetric=False
)

So easy, so wrong ... but what's the problem here? It's a bit subtle and not easy to spot:

> We actually don't make a multi-step-ahead forecast up to the 36th step ahead. Instead, we make 36 single-step-ahead forecasts always using the most recent data. But that's a solution to a different learning task!

To fix this problem, we could write some code to do this recursively as in the M4 competition:

In [ ]:
# slightly modified code from the M4 study
predictions = []
last_window = x_train[-1, :].reshape(1, -1)  # make it into 2d array

last_prediction = model.predict(last_window)[0]  # take value from array

for i in range(len(fh)):
    # append prediction
    predictions.append(last_prediction)

    # update last window using previously predicted value
    last_window[0] = np.roll(last_window[0], -1)
    last_window[0, (len(last_window[0]) - 1)] = last_prediction

    # predict next step ahead
    last_prediction = model.predict(last_window)[0]

y_pred_rec = pd.Series(predictions, index=y_test.index)

In [ ]:
from aeon.performance_metrics.forecasting import mean_absolute_percentage_error

mean_absolute_percentage_error(
    y_test, pd.Series(y_pred_rec, index=y_test.index), symmetric=False
)

To summarize the potential pitfalls here:

> Obtaining regressor predictions and converting them back into forecasts is non-trivial and error prone:
> * some boilerplate code needs to be written, which just as in pitfall no.2 introduces potential for problems;
> * It isn't exactly obvious that this boilerplate code had to be written in the first place, creating a subtle failure point.

### How does `aeon` help avoid the above pitfalls?

`aeon` mitigates the above pitfalls with:

* Its unified interface for forecasters - any strategy to produce forecasts is a forecaster. Through the unified interface, forecasters are directly compatible with deployment and evaluation workflows appropriate for forecasters;
* Its declarative specification interface that minimizes boilerplate code - it's minimized to the bare necessities to tell `aeon` which forecaster you want to build.

Nevertheless, `aeon` aims to be flexible, and tries to avoid to railroad the user into specific methodological choices.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

from aeon.forecasting.compose import make_reduction

In [ ]:
# declarative forecaster specification - just two lines!
regressor = KNeighborsRegressor(n_neighbors=1)
forecaster = make_reduction(regressor, window_length=15, strategy="recursive")

In [ ]:
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)

... and that's it!

Note that there is no `x_train` or other boilerplate artefacts, since construction of the lagged features and other boilerplate code are taken care of by the forecaster internally.

For more details on the `aeon` composition interface, refer to Section 3 of the main forecasting tutorial.